In [1]:
import pandas as pd
import numpy as np

from sklearn import mixture
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import itertools

In [2]:
players = pd.read_csv('merged_players42-12.csv',index_col=0,parse_dates=['debut','finalGame'])
players.head()

,playerID,FPCT,POS,G,SH,OBP,AB/G,HR/G,SB/G,RBI/G,IBB/G,SF/G,debut,finalGame,weight,servingDays,debutAge,inducted
0,aaronha01,0.979124,OF,3298,21.0,0.373949,3.748939,0.228927,0.072771,0.696483,0.088842,0.036689,1954-04-13,1976-10-03,180.0,8209.0,20.293151,Y
1,aaronto01,0.982705,1B,437,9.0,0.291506,2.160183,0.029748,0.020595,0.215103,0.006865,0.013730,1962-04-10,1971-09-26,190.0,3456.0,23.287671,N
2,aasedo01,0.955399,P,448,0.0,0.000000,0.011161,0.000000,0.000000,0.000000,0.000000,0.000000,1977-07-26,1990-10-03,190.0,4817.0,23.580822,N
3,abadan01,0.986486,1B,15,0.0,0.240000,1.400000,0.000000,0.000000,0.000000,0.000000,0.000000,2001-09-10,2006-04-13,184.0,1676.0,29.712329,N
4,abbotje01,0.975208,OF,233,5.0,0.307453,2.557940,0.077253,0.025751,0.356223,0.008584,0.030043,1997-06-10,2001-09-29,190.0,1572.0,25.457534,N


In [3]:
players = players[players['finalGame'].apply(lambda x: x.year<=1960)]

In [6]:
players.columns

Index(['playerID', 'FPCT', 'POS', 'G', 'SH', 'OBP', 'AB/G', 'HR/G', 'SB/G',
       'RBI/G', 'IBB/G', 'SF/G', 'debut', 'finalGame', 'weight', 'servingDays',
       'debutAge', 'inducted'],
      dtype='object')

In [12]:
feat_col = ['FPCT', 'POS', 'G', 'SH', 'OBP', 'AB/G', 'HR/G', 'SB/G',
       'RBI/G', 'IBB/G', 'SF/G','weight', 'servingDays',
       'debutAge']
def PowerSetsRecursive2(items):
    # the power set of the empty set has one element, the empty set
    result = [[]]
    for x in items:
        result.extend([subset + [x] for subset in result])
    return result
features = PowerSetsRecursive2(feat_col)

In [13]:
# features = [['G'],
#             ['OBP'],
#             ['FPCT'],
#             ['FPCT','POS'],
#             ['OBP','FPCT','POS'],
#             ['G','OBP'],
#             ['G','OBP','FPCT'],
#             ['G','OBP','FPCT','POS'],
#             ['G','FPCT'],
#             ['G','FPCT','POS'],
#             ['FPCT', 'POS', 'G', 'SH', 'OBP', 'AB/G', 'HR/G', 'SB/G',
#        'RBI/G', 'IBB/G', 'SF/G', 'weight', 'debutAge']]

num_list = []
for feature in features:
    if (len(feature)==0 or len(feature)>=3):
        continue
    X = players[feature]
    if ('POS' in feature):
        X = pd.get_dummies(X, columns = ['POS'] )
    y = players[['inducted']]
    
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 10)
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type='full', random_state = 2018)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
            
    loglike=best_gmm.score_samples(X)
    players_pre = players[['playerID','servingDays','inducted']]
    players_pre.loc[:,'loglike']=loglike

    player10 = players_pre[players_pre['servingDays']>2000].sort_values(['loglike'],ascending=True).iloc[:10,:]
    num = (player10[player10['inducted']=='Y'].shape[0])
    num_list.append([num,feature])
    print(num)
    if num>=3:
        print(feature)
        print(players_pre[players_pre['servingDays']>2000].sort_values('loglike'))

num_list

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0
1
1
3
['G']
       playerID  servingDays inducted    loglike
2087  ennisde01       4795.0        N -12.374782
5382  pafkoan01       5849.0        N -12.043211
1671   darkal01       5194.0        N -11.894830
2400  furilca01       5135.0        N -11.763116
3679   kellge01       5100.0        Y -11.698802
7163  thomsbo01       5060.0        N -11.607090
4181  lockmwh01       5468.0        N -11.021345
1884   dobyla01       4404.0        Y -10.471012
3744  kinerra01       3449.0        Y -10.268910
4253  lowrepe01       4886.0        N -10.073513
6071  robinja02       3456.0        Y -10.028491
710   boonera01       4360.0        N -10.008236
4617  mcdougi01       3453.0        N  -9.932193
1165  carrach01       3445.0        N  -9.911828
3247  holmeto01       3820.0        N  -9.902910
6067  robined01       5485.0        N  -9.894204
2995  hattogr01       5282.0        N  -9.889083
6506  seminan01       5121.0        N  -9.875798
267   avilabo01       3804.0        N  -9.869359
4776  

2
2
4
['G', 'HR/G']
       playerID  servingDays inducted    loglike
6184  rosenal01       3308.0        N -12.724145
3744  kinerra01       3449.0        Y -11.088482
1884   dobyla01       4404.0        Y -10.689764
1102  camparo01       3449.0        Y -10.097370
4181  lockmwh01       5468.0        N -10.011890
7910  zernigu01       3811.0        N  -9.638537
6489  seerepa01       2159.0        N  -9.498418
7163  thomsbo01       5060.0        N  -9.312909
2087  ennisde01       4795.0        N  -9.250657
3679   kellge01       5100.0        Y  -9.225548
7147  thompha02       3363.0        N  -8.993100
4253  lowrepe01       4886.0        N  -8.985652
4219  lopatst01       4284.0        N  -8.765588
7590  westlwa01       3315.0        N  -8.760436
1671   darkal01       5194.0        N  -8.614830
3401  irvinmo01       2641.0        Y  -8.565689
2252  fitzged01       4166.0        N  -8.331786
6976  sullijo06       2674.0        N  -8.318705
5566  peskyjo01       4546.0        N  -8.277890


2
1
1
1
2
2
1
1
3
['HR/G', 'SB/G']
       playerID  servingDays inducted    loglike
6071  robinja02       3456.0        Y  -0.819450
3744  kinerra01       3449.0        Y  -0.330559
7268  tucketh01       3302.0        N   0.032264
2288  fondyde01       2714.0        N   0.157814
6914  stirnsn01       3299.0        N   0.228449
6184  rosenal01       3308.0        N   0.586308
1392   coangi01       3651.0        N   0.610040
7910  zernigu01       3811.0        N   0.760204
1102  camparo01       3449.0        Y   0.863511
5290   olmolu01       2875.0        N   0.984539
455   beardte01       3537.0        N   1.170726
6241  russeji01       3306.0        N   1.184490
3099  hermage01       3692.0        N   1.220813
267   avilabo01       3804.0        N   1.266075
1884   dobyla01       4404.0        Y   1.278330
638   blattbu01       2724.0        N   1.284873
3401  irvinmo01       2641.0        Y   1.375297
7147  thompha02       3363.0        N   1.378124
3423  jacksra01       3433.0      

4
['G', 'RBI/G']
       playerID  servingDays inducted    loglike
1102  camparo01       3449.0        Y -11.711452
1884   dobyla01       4404.0        Y -11.405998
6184  rosenal01       3308.0        N -11.342292
4181  lockmwh01       5468.0        N -10.749127
3744  kinerra01       3449.0        Y -10.694858
4253  lowrepe01       4886.0        N -10.637575
3401  irvinmo01       2641.0        Y -10.597404
7910  zernigu01       3811.0        N -10.571151
1671   darkal01       5194.0        N -10.393355
2087  ennisde01       4795.0        N  -9.935762
31    adamsbo03       4754.0        N  -9.864254
4790  miksied01       5214.0        N  -9.814225
3409  jablora01       2713.0        N  -9.741644
7163  thomsbo01       5060.0        N  -9.579161
5290   olmolu01       2875.0        N  -9.567809
4856  miranwi01       3046.0        N  -9.560929
3828  konstji01       4476.0        N  -9.437340
2464  gardeda01       2167.0        N  -9.349836
7590  westlwa01       3315.0        N  -9.346593
742

3
['AB/G', 'RBI/G']
       playerID  servingDays inducted   loglike
1102  camparo01       3449.0        Y -4.573493
3744  kinerra01       3449.0        Y -4.213575
2087  ennisde01       4795.0        N -4.185707
6184  rosenal01       3308.0        N -4.158440
5626  piconma01       2462.0        N -3.276913
1884   dobyla01       4404.0        Y -3.063461
7910  zernigu01       3811.0        N -3.022110
5951  rhodedu01       2630.0        N -2.494467
1671   darkal01       5194.0        N -2.464346
5290   olmolu01       2875.0        N -2.357846
2400  furilca01       5135.0        N -2.320381
7163  thomsbo01       5060.0        N -2.205669
3401  irvinmo01       2641.0        Y -2.204315
7867  youngbo01       3681.0        N -2.000190
7590  westlwa01       3315.0        N -1.945949
1462  connebi01       2855.0        N -1.867061
6190   rossbo01       2153.0        N -1.861080
3247  holmeto01       3820.0        N -1.758127
6067  robined01       5485.0        N -1.756365
3679   kellge01     

1
0
3
['G', 'IBB/G']
       playerID  servingDays inducted    loglike
4219  lopatst01       4284.0        N -12.073356
1102  camparo01       3449.0        Y -10.212040
7750  wilsore01       3290.0        N  -9.090812
1884   dobyla01       4404.0        Y  -8.487359
150   amorosa01       2963.0        N  -8.461750
6626  simpsha01       3081.0        N  -8.308845
531   berbelo01       2200.0        N  -8.234664
3744  kinerra01       3449.0        Y  -7.907739
710   boonera01       4360.0        N  -7.465330
5382  pafkoan01       5849.0        N  -7.463373
5196  norenir01       3819.0        N  -7.317852
3679   kellge01       5100.0        Y  -7.288210
2400  furilca01       5135.0        N  -7.184477
2087  ennisde01       4795.0        N  -7.124257
7463  walkeru01       3708.0        N  -7.113801
243   astrojo01       3926.0        N  -6.989775
3654   kattra01       2499.0        N  -6.941384
4181  lockmwh01       5468.0        N  -6.689094
5951  rhodedu01       2630.0        N  -6.679585

2
3
['RBI/G', 'IBB/G']
       playerID  servingDays inducted    loglike
1102  camparo01       3449.0        Y  -4.122941
4219  lopatst01       4284.0        N  -2.178744
6184  rosenal01       3308.0        N  -0.470377
531   berbelo01       2200.0        N  -0.452425
150   amorosa01       2963.0        N  -0.395693
2087  ennisde01       4795.0        N  -0.342217
7750  wilsore01       3290.0        N  -0.224313
1884   dobyla01       4404.0        Y   0.260077
2400  furilca01       5135.0        N   0.496647
3401  irvinmo01       2641.0        Y   0.860420
7163  thomsbo01       5060.0        N   0.932554
6661  sleatlo01       3078.0        N   1.062125
6626  simpsha01       3081.0        N   1.169154
710   boonera01       4360.0        N   1.182360
3409  jablora01       2713.0        N   1.243264
3425  jacksro01       2161.0        N   1.257805
6067  robined01       5485.0        N   1.267500
6071  robinja02       3456.0        Y   1.336671
3744  kinerra01       3449.0        Y   1.3826

1
2
1
0
0
0
0
0
2
1
0
0
2
1
3
['RBI/G', 'debutAge']
       playerID  servingDays inducted   loglike
2087  ennisde01       4795.0        N -7.273664
3401  irvinmo01       2641.0        Y -6.406781
6399  scheica01       3913.0        N -6.387816
4776  michaca01       4026.0        N -6.333641
6184  rosenal01       3308.0        N -6.322106
3744  kinerra01       3449.0        Y -6.241824
923   brownto02       3340.0        N -5.789290
7910  zernigu01       3811.0        N -5.355391
1062  cabalpu01       2935.0        N -5.296078
1884   dobyla01       4404.0        Y -5.211248
1102  camparo01       3449.0        Y -4.806848
3294  houttar01       4529.0        N -4.719421
6071  robinja02       3456.0        Y -4.613167
5290   olmolu01       2875.0        N -4.513098
680   bolgeji01       3373.0        N -4.493229
2400  furilca01       5135.0        N -4.478771
797   brancra01       4470.0        N -4.478246
5396  palicer01       4174.0        N -4.478177
4790  miksied01       5214.0        

[[0, ['FPCT']],
 [1, ['POS']],
 [1, ['FPCT', 'POS']],
 [3, ['G']],
 [0, ['FPCT', 'G']],
 [2, ['POS', 'G']],
 [2, ['SH']],
 [0, ['FPCT', 'SH']],
 [1, ['POS', 'SH']],
 [3, ['G', 'SH']],
 [2, ['OBP']],
 [0, ['FPCT', 'OBP']],
 [1, ['POS', 'OBP']],
 [1, ['G', 'OBP']],
 [1, ['SH', 'OBP']],
 [1, ['AB/G']],
 [0, ['FPCT', 'AB/G']],
 [2, ['POS', 'AB/G']],
 [0, ['G', 'AB/G']],
 [1, ['SH', 'AB/G']],
 [2, ['OBP', 'AB/G']],
 [3, ['HR/G']],
 [2, ['FPCT', 'HR/G']],
 [2, ['POS', 'HR/G']],
 [4, ['G', 'HR/G']],
 [4, ['SH', 'HR/G']],
 [4, ['OBP', 'HR/G']],
 [2, ['AB/G', 'HR/G']],
 [1, ['SB/G']],
 [1, ['FPCT', 'SB/G']],
 [1, ['POS', 'SB/G']],
 [2, ['G', 'SB/G']],
 [2, ['SH', 'SB/G']],
 [1, ['OBP', 'SB/G']],
 [1, ['AB/G', 'SB/G']],
 [3, ['HR/G', 'SB/G']],
 [4, ['RBI/G']],
 [0, ['FPCT', 'RBI/G']],
 [3, ['POS', 'RBI/G']],
 [4, ['G', 'RBI/G']],
 [3, ['SH', 'RBI/G']],
 [4, ['OBP', 'RBI/G']],
 [3, ['AB/G', 'RBI/G']],
 [3, ['HR/G', 'RBI/G']],
 [4, ['SB/G', 'RBI/G']],
 [1, ['IBB/G']],
 [1, ['FPCT', 'IBB/G']],
 [0,

In [14]:
players[players['inducted']=='Y']

,playerID,FPCT,POS,G,SH,OBP,AB/G,HR/G,SB/G,RBI/G,IBB/G,SF/G,debut,finalGame,weight,servingDays,debutAge,inducted
927,brownwi02,1.000000,OF,21,0.0,0.179104,3.190476,0.047619,0.095238,0.285714,0.000000,0.000000,1947-07-19,1947-08-17,200.0,29.0,32.567123,Y
1102,camparo01,0.987926,C,1215,30.0,0.360217,3.460905,0.199177,0.020576,0.704527,0.024691,0.014815,1948-04-20,1957-09-29,190.0,3449.0,27.317808,Y
1884,dobyla01,0.979901,OF,1533,20.0,0.385864,3.488584,0.165036,0.030659,0.632746,0.012394,0.016308,1947-07-05,1959-07-26,180.0,4404.0,24.523288,Y
3401,irvinmo01,0.985971,OF,764,9.0,0.383148,3.270942,0.129581,0.036649,0.579843,0.006545,0.014398,1949-07-08,1956-09-30,195.0,2641.0,30.536986,Y
3679,kellge01,0.969984,3B,1795,149.0,0.367394,3.733705,0.043454,0.028412,0.484680,0.003900,0.011142,1943-09-28,1957-09-14,175.0,5100.0,21.753425,Y
3744,kinerra01,0.975323,OF,1472,9.0,0.397951,3.536005,0.250679,0.014946,0.689538,0.000679,0.004755,1946-04-16,1955-09-25,195.0,3449.0,24.304110,Y
6071,robinja02,0.988926,2B,1382,104.0,0.408915,3.528944,0.099132,0.142547,0.531114,0.005065,0.006512,1947-04-15,1956-09-30,195.0,3456.0,28.304110,Y


In [15]:
players[players['playerID']=='pafkoan01']

,playerID,FPCT,POS,G,SH,OBP,AB/G,HR/G,SB/G,RBI/G,IBB/G,SF/G,debut,finalGame,weight,servingDays,debutAge,inducted
5382,pafkoan01,0.984904,OF,1852,82.0,0.350374,3.397408,0.115011,0.020518,0.526998,0.00378,0.008099,1943-09-24,1959-09-29,190.0,5849.0,22.742466,N


In [27]:
players[players['playerID']=='rosenal01']

,playerID,FPCT,POS,G,SH,OBP,AB/G,HR/G,SB/G,RBI/G,IBB/G,SF/G,debut,finalGame,weight,servingDays,debutAge,inducted
6184,rosenal01,0.968578,3B,1044,9.0,0.384192,3.568008,0.183908,0.037356,0.686782,0.008621,0.024904,1947-09-10,1956-09-30,180.0,3308.0,23.706849,N


In [16]:
hof = pd.read_csv("HallOfFame.csv")

In [18]:
hof[hof['playerID']=='brownwi02']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
3783,brownwi02,2006,Negro League,NaN,NaN,NaN,Y,Player,NaN


In [19]:
hof[hof['playerID']=='camparo01']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
1973,camparo01,1964,Run Off,201.0,NaN,138.0,N,Player,1st
1975,camparo01,1964,BBWAA,201.0,151.0,115.0,N,Player,NaN
2066,camparo01,1966,BBWAA,302.0,227.0,197.0,N,Player,NaN
2118,camparo01,1967,BBWAA,292.0,219.0,204.0,N,Player,NaN
2119,camparo01,1967,Run Off,306.0,NaN,170.0,N,Player,1st
2195,camparo01,1968,BBWAA,283.0,213.0,205.0,N,Player,NaN
2245,camparo01,1969,BBWAA,340.0,255.0,270.0,Y,Player,NaN


In [20]:
hof[hof['playerID']=='dobyla01']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
2094,dobyla01,1966,BBWAA,302.0,227.0,7.0,N,Player,NaN
2165,dobyla01,1967,BBWAA,292.0,219.0,10.0,N,Player,NaN
2190,dobyla01,1967,Run Off,306.0,NaN,1.0,N,Player,1st
3531,dobyla01,1998,Veterans,NaN,NaN,NaN,Y,Player,NaN


In [21]:
hof[hof['playerID']=='irvinmo01']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
2494,irvinmo01,1973,Negro League,NaN,NaN,NaN,Y,Player,NaN


In [22]:
hof[hof['playerID']=='kellge01']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
1992,kellge01,1964,BBWAA,201.0,151.0,33.0,N,Player,NaN
2026,kellge01,1964,Run Off,201.0,NaN,8.0,N,Player,1st
2085,kellge01,1966,BBWAA,302.0,227.0,29.0,N,Player,NaN
2141,kellge01,1967,BBWAA,292.0,219.0,40.0,N,Player,NaN
2164,kellge01,1967,Run Off,306.0,NaN,11.0,N,Player,1st
2210,kellge01,1968,BBWAA,283.0,213.0,47.0,N,Player,NaN
2261,kellge01,1969,BBWAA,340.0,255.0,60.0,N,Player,NaN
2301,kellge01,1970,BBWAA,300.0,225.0,90.0,N,Player,NaN
2351,kellge01,1971,BBWAA,360.0,270.0,105.0,N,Player,NaN
2407,kellge01,1972,BBWAA,396.0,297.0,115.0,N,Player,NaN


In [23]:
hof[hof['playerID']=='kinerra01']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
1835,kinerra01,1960,BBWAA,269.0,202.0,3.0,N,Player,NaN
1918,kinerra01,1962,BBWAA,160.0,120.0,5.0,N,Player,NaN
1994,kinerra01,1964,BBWAA,201.0,151.0,31.0,N,Player,NaN
2039,kinerra01,1964,Run Off,201.0,NaN,3.0,N,Player,1st
2074,kinerra01,1966,BBWAA,302.0,227.0,74.0,N,Player,NaN
2121,kinerra01,1967,BBWAA,292.0,219.0,124.0,N,Player,NaN
2140,kinerra01,1967,Run Off,306.0,NaN,41.0,N,Player,1st
2198,kinerra01,1968,BBWAA,283.0,213.0,118.0,N,Player,NaN
2247,kinerra01,1969,BBWAA,340.0,255.0,137.0,N,Player,NaN
2293,kinerra01,1970,BBWAA,300.0,225.0,167.0,N,Player,NaN


In [24]:
hof[hof['playerID']=='robinja02']

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
1885,robinja02,1962,BBWAA,160.0,120.0,124.0,Y,Player,NaN


In [25]:
hof[hof['playerID']=='rosenal01'] 

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note


In [26]:
X = players[['G','HR/G']]
y = players[['inducted']]

lowest_bic = np.infty
bic = []
n_components_range = range(1, 10)
for n_components in n_components_range:
    # Fit a Gaussian mixture with EM
    gmm = mixture.GaussianMixture(n_components=n_components,
                                  covariance_type='full', random_state = 2018)
    gmm.fit(X)
    bic.append(gmm.bic(X))
    if bic[-1] < lowest_bic:
        lowest_bic = bic[-1]
        best_gmm = gmm

loglike=best_gmm.score_samples(X)
players_pre = players[['playerID','servingDays','G','HR/G','inducted']]
players_pre.loc[:,'loglike']=loglike

player10 = players_pre[players_pre['servingDays']>2000].sort_values(['loglike'],ascending=True).iloc[:10,:]
num = (player10[player10['inducted']=='Y'].shape[0])
num_list.append([num,feature])
print(num)
if num>=3:
    print(feature)
    print(players_pre[players_pre['servingDays']>2000].sort_values('loglike'))

4
['FPCT', 'POS', 'G', 'SH', 'OBP', 'AB/G', 'HR/G', 'SB/G', 'RBI/G', 'IBB/G', 'SF/G', 'weight', 'servingDays', 'debutAge']
       playerID  servingDays     G      HR/G inducted    loglike
6184  rosenal01       3308.0  1044  0.183908        N -12.724145
3744  kinerra01       3449.0  1472  0.250679        Y -11.088482
1884   dobyla01       4404.0  1533  0.165036        Y -10.689764
1102  camparo01       3449.0  1215  0.199177        Y -10.097370
4181  lockmwh01       5468.0  1666  0.068427        N -10.011890
7910  zernigu01       3811.0  1234  0.192058        N  -9.638537
6489  seerepa01       2159.0   561  0.153298        N  -9.498418
7163  thomsbo01       5060.0  1779  0.148398        N  -9.312909
2087  ennisde01       4795.0  1903  0.151340        N  -9.250657
3679   kellge01       5100.0  1795  0.043454        Y  -9.225548
7147  thompha02       3363.0   933  0.138264        N  -8.993100
4253  lowrepe01       4886.0  1401  0.026410        N  -8.985652
4219  lopatst01       4284.0   8

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
